<a href="https://colab.research.google.com/github/Seghelicious/Vayner-Box/blob/master/Vayner_Box_Asset_Portfolio_Returns%2C_Regression_and_Monte_Carlo_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mean - Variance Optimisation**

The aim of this project is to build asset portfolios from the following funds:

- AXA Money Market Fund
- Stanbic ETF30 Fund
- Stanbic Guranteed Income Fund
- Stanbic Absolute Fund
- Stanbic Eurobond Fund
- FBN Eurobond Fund

1. First of all, I calculated the monthly returns of each of the funds.
2. Then I built six asset portfolio classes for investors with  different risk profiles (Conservative, Cautious, Balanced, Growth, Aggressive and Dollar)
3. I estimated the relationship between each of the funds and 16 Macroeconomic variables and bond-yields using the **Multivariate Linear Regresion** model.
4. I modelled a Monte Carlo simulation to predict the likelihood that a fund would make a certain at a given time in future.




In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Import the required libraries and packages

import pandas as pd
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import itertools
import math
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import datetime
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

pd.options.display.max_rows = None
pd.options.display.max_columns = None

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
fund_prices = pd.read_csv('/content/drive/My Drive/Python University/Vayner Box/fund_prices (version 1).csv')
axa_mmf_ = pd.read_csv('/content/drive/My Drive/Python University/Vayner Box/axa_mmf1.csv')

In [0]:
fund_prices['Date'] = pd.to_datetime(fund_prices.Date, format = '%Y-%m-%d') 
axa_mmf_['Date'] = pd.to_datetime(axa_mmf_.Date, format = '%Y-%m-%d')

In [0]:
axa_mmf_['AXA Money Market Fund Yield'] = (1 + axa_mmf_['AXA Money Market Fund Yield']) ** (1/365) - 1

In [0]:
axa_mmf_['month year'] = axa_mmf_['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [0]:
axa_mmf_['month year'] = axa_mmf_['Date'].apply(lambda x: x.strftime('%Y-%m'))
axa_mmf_ = axa_mmf_.drop(columns = ['Date'], axis = 1)

In [0]:
axa_mmf_ = axa_mmf_.groupby('month year')['AXA Money Market Fund Yield'].sum().reset_index()

#### **Monthly Returns**

In [0]:
fund_prices = fund_prices.copy()

In [0]:
# extract some date information

fund_prices['year'] = fund_prices['Date'].dt.year
fund_prices['month'] = fund_prices['Date'].dt.month
fund_prices['day'] = fund_prices['Date'].dt.day
fund_prices['day of month'] = fund_prices['Date'].dt.daysinmonth

In [0]:
first_day28 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 28)]
last_day28 = fund_prices[(fund_prices['day'] == 28) & (fund_prices['day of month'] == 28)]
first_day30 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 30)]
last_day30 = fund_prices[(fund_prices['day'] == 30) & (fund_prices['day of month'] == 30)]
first_day31 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 31)]
last_day31 = fund_prices[(fund_prices['day'] == 31) & (fund_prices['day of month'] == 31)]

In [0]:
new_prices = pd.concat([first_day28, last_day28, first_day30, last_day30, first_day31, last_day31], axis = 0)

In [0]:
new_prices['month year'] = new_prices['Date'].apply(lambda x: x.strftime('%Y-%m'))
new_prices = new_prices.sort_index(ascending = True)

In [0]:
stb_etf_returns = new_prices['Stanbic ETF 30'].div(new_prices.groupby('month year')['Stanbic ETF 30'].shift(1)) - 1
stb_abs_returns = new_prices['Stanbic Absolute Fund'].div(new_prices.groupby('month year')['Stanbic Absolute Fund'].shift(1)) - 1
stb_gtd_returns = new_prices['Stanbic Guaranteed Income Fund'].div(new_prices.groupby('month year')['Stanbic Guaranteed Income Fund'].shift(1)) - 1
stb_euro_returns = new_prices['Stanbic Eurobond Fund'].div(new_prices.groupby('month year')['Stanbic Eurobond Fund'].shift(1)) - 1
fbn_euro_returns = new_prices['FBN Eurobond Fund'].div(new_prices.groupby('month year')['FBN Eurobond Fund'].shift(1)) - 1

In [0]:
monthly_returns = pd.DataFrame()

monthly_returns['Date'] = new_prices['month year']
monthly_returns['Stanbic ETF 30'] = stb_etf_returns
monthly_returns['Stanbic Absolute Fund'] = stb_abs_returns
monthly_returns['Stanbic Guaranteed Income Fund'] = stb_gtd_returns
monthly_returns['Stanbic Eurobond Fund'] = stb_euro_returns
monthly_returns['FBN Eurobond Fund'] = fbn_euro_returns
#monthly_returns['AXA Money Market Fund Yield'] = axa_mmf_['AXA Money Market Fund Yield']

In [0]:
monthly_returns = monthly_returns.dropna()
monthly_returns = monthly_returns.reset_index(drop = True, inplace = False)
month_returns = pd.concat([monthly_returns, axa_mmf_['AXA Money Market Fund Yield']], axis = 1)

#### **Portfolio Returns**

In [0]:
returns1 = month_returns[['Stanbic Absolute Fund', 'AXA Money Market Fund Yield']]
returns2 = month_returns[['Stanbic Absolute Fund', 'Stanbic Guaranteed Income Fund', 'AXA Money Market Fund Yield']]            
returns3 = month_returns[['Stanbic Absolute Fund', 'Stanbic ETF 30']]
returns4 = month_returns[['Stanbic Eurobond Fund', 'FBN Eurobond Fund']]

In [0]:
axa_mmf1, axa_mmf2, axa_mmf3 = [0.9, 0.72, 0.13]
stb_abs1, stb_abs2, stb_abs3, stb_abs4, stb_abs5 = [0.1, 0.24, 0.87, 0.73, 0.51]
stb_GI = 0.04
stb_etf1, stb_etf2 = [0.27, 0.49]
stb_euro, fbn_euro = [0.5, 0.5]

In [0]:
conserv = np.array([stb_abs1, axa_mmf1])
caut = np.array([stb_abs2, stb_GI, axa_mmf2])
bal = np.array([stb_abs3, axa_mmf3])
grow = np.array([stb_abs4, stb_etf1])
aggres = np.array([stb_abs5, stb_etf2])
doll = np.array([stb_euro, fbn_euro])

In [0]:
conservative = np.dot(returns1, conserv)
cautious = np.dot(returns2, caut)
balanced = np.dot(returns1, bal)
growth = np.dot(returns3, grow)
aggressive = np.dot(returns3, aggres)
dollar = np.dot(returns4, doll)

In [0]:
portfolios = pd.DataFrame()

portfolios['Date'] = month_returns['Date']
portfolios['Conservative'] = conservative
portfolios['Cautious'] = cautious
portfolios['Balanced'] = balanced
portfolios['Growth'] = growth
portfolios['Aggressive'] = aggressive
portfolios['Dollar'] = dollar

In [22]:
portfolios.head()

,Date,Conservative,Cautious,Balanced,Growth,Aggressive,Dollar
0,2016-08,0.006032,0.006028,0.006457,0.004061,0.002050,0.005263
1,2016-09,0.006469,0.006824,0.008787,0.007953,0.006955,-0.000189
2,2016-10,0.009715,0.010810,0.014668,0.000713,-0.011338,-0.014401
3,2016-11,0.011753,0.012317,0.014148,-0.006651,-0.023928,-0.002098
4,2016-12,0.012194,0.012592,0.013947,0.024315,0.032521,0.010129


#### **Regression**

In [0]:
col_names = ['Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves', 'USD/NGN (BDC)', 
             'USD/NGN (I&E)', '91-Day Yield', '182-Day Yield', '364-Day Yield', 
             '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield', '7yr-Bond Yield', 
             '10yr-Bond Yield', '12yr-Bond Yield', '15yr-Bond Yield', '20yr-Bond Yield']

macro_data = pd.read_csv('/content/drive/My Drive/Python University/Vayner Box/Portfolios & Macro Data.csv', usecols = col_names)

In [24]:
data = pd.concat([portfolios, macro_data], axis = 1)
data.columns

Index(['Date', 'Conservative', 'Cautious', 'Balanced', 'Growth', 'Aggressive',
       'Dollar', 'Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves',
       'USD/NGN (BDC)', 'USD/NGN (I&E)', '91-Day Yield', '182-Day Yield',
       '364-Day Yield', '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield',
       '7yr-Bond Yield', '10yr-Bond Yield', '12yr-Bond Yield',
       '15yr-Bond Yield', '20yr-Bond Yield'],
      dtype='object')

In [0]:
# Independent Variables
X1 = data[['Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves', 'USD/NGN (BDC)', 'USD/NGN (I&E)', '91-Day Yield', 
           '182-Day Yield', '364-Day Yield', '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield', '7yr-Bond Yield', 
           '10yr-Bond Yield', '12yr-Bond Yield', '15yr-Bond Yield', '20yr-Bond Yield']]

##### *Conservative Portfolio*

In [26]:
# Dependent Variable
Y1 = data['Conservative']

X1a = sm.add_constant(X1)
reg1 = sm.OLS(Y1, X1a).fit()
#prediction1 = reg1.predict(X1)

reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Conservative   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     1.621
Date:                Wed, 22 Apr 2020   Prob (F-statistic):              0.220
Time:                        19:38:18   Log-Likelihood:                 148.26
No. Observations:                  28   AIC:                            -260.5
Df Residuals:                      10   BIC:                            -236.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0107      0.001     19.743      0.000       0.010       0.012
Oil Prices          0.0061      0.008      0.775      0.456      -0.011       0.024
Inflation          -0.0051      0.018     -0.285      0.782      -0.045       0.035
NSE ASI             0.0207      0.008      2.731      0.021       0.004       0.038
FX Reserves         0.0251      0.013      1.986      0.075      -0.003       0.053
USD/NGN (BDC)      -0.0189      0.011     -1.671      0.126      -0.044       0.006
USD/NGN (I&E)       0.0094      0.010      0.914      0.382      -0.013       0.032
91-Day Yield        0.0006      0.005      0.117      0.909      -0.011       0.013
182-Day Yield      -0.0033      0.009     -0.352      0.732      -0.024       0.018
364-Day Yield       0.0007      0.012      0.058      0.955      -0.026       0.027
2yr-Bond Yield     -0.0062      0.015     -0.410      0.690      -0.040       0.027
3yr-Bond Yield      0.0208      0.031      0.674      0.516      -0.048       0.089
5-yr-Bond Yield     0.0049      0.048      0.102      0.920      -0.102       0.112
7yr-Bond Yield     -0.0199      0.072     -0.277      0.787      -0.180       0.140
10yr-Bond Yield    -0.0041      0.087     -0.047      0.963      -0.197       0.189
12yr-Bond Yield     0.0663      0.160      0.415      0.687      -0.290       0.422
15yr-Bond Yield    -0.0406      0.112     -0.362      0.725      -0.291       0.210
20yr-Bond Yield    -0.0209      0.058     -0.358      0.728      -0.151       0.109
==============================================================================
Omnibus:                        4.125   Durbin-Watson:                   1.533
Prob(Omnibus):                  0.127   Jarque-Bera (JB):                2.519
Skew:                           0.500   Prob(JB):                        0.284
Kurtosis:                       4.076   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
# Independent Variables
X1_conserv = data[['NSE ASI', 'FX Reserves']]

# Dependent Variable
Y1_conserv = data['Conservative']

X1a_conserv = sm.add_constant(X1_conserv)
reg1_conserv = sm.OLS(Y1_conserv, X1a_conserv).fit()
#prediction1 = reg1.predict(X1)

reg1_conserv.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Conservative   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     8.632
Date:                Wed, 22 Apr 2020   Prob (F-statistic):            0.00141
Time:                        19:38:18   Log-Likelihood:                 137.08
No. Observations:                  28   AIC:                            -268.2
Df Residuals:                      25   BIC:                            -264.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0110      0.000     27.597      0.000       0.010       0.012
NSE ASI         0.0166      0.006      2.866      0.008       0.005       0.029
FX Reserves     0.0250      0.008      2.961      0.007       0.008       0.042
==============================================================================
Omnibus:                        1.794   Durbin-Watson:                   1.029
Prob(Omnibus):                  0.408   Jarque-Bera (JB):                1.288
Skew:                          -0.522   Prob(JB):                        0.525
Kurtosis:                       2.882   Cond. No.                         23.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Cautious Portfolio*

In [28]:
Y2 = data['Cautious']

X1b = sm.add_constant(X1)
reg2 = sm.OLS(Y2, X1b).fit()
#prediction1 = reg1.predict(X1)

reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Cautious   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     1.701
Date:                Wed, 22 Apr 2020   Prob (F-statistic):              0.197
Time:                        19:38:18   Log-Likelihood:                 148.24
No. Observations:                  28   AIC:                            -260.5
Df Residuals:                      10   BIC:                            -236.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0109      0.001     20.009      0.000       0.010       0.012
Oil Prices          0.0049      0.008      0.621      0.549      -0.013       0.023
Inflation          -0.0026      0.018     -0.143      0.889      -0.042       0.037
NSE ASI             0.0211      0.008      2.787      0.019       0.004       0.038
FX Reserves         0.0257      0.013      2.036      0.069      -0.002       0.054
USD/NGN (BDC)      -0.0195      0.011     -1.724      0.115      -0.045       0.006
USD/NGN (I&E)       0.0098      0.010      0.954      0.363      -0.013       0.033
91-Day Yield        0.0013      0.005      0.237      0.817      -0.011       0.013
182-Day Yield      -0.0034      0.009     -0.360      0.726      -0.024       0.017
364-Day Yield      -0.0033      0.012     -0.277      0.788      -0.030       0.023
2yr-Bond Yield     -0.0077      0.015     -0.513      0.619      -0.041       0.026
3yr-Bond Yield      0.0283      0.031      0.918      0.380      -0.040       0.097
5-yr-Bond Yield     0.0119      0.048      0.247      0.810      -0.095       0.119
7yr-Bond Yield     -0.0443      0.072     -0.617      0.551      -0.204       0.116
10yr-Bond Yield    -0.0093      0.087     -0.107      0.917      -0.203       0.184
12yr-Bond Yield     0.0452      0.160      0.283      0.783      -0.311       0.401
15yr-Bond Yield    -0.0144      0.112     -0.128      0.900      -0.265       0.236
20yr-Bond Yield    -0.0040      0.058     -0.068      0.947      -0.134       0.126
==============================================================================
Omnibus:                        4.636   Durbin-Watson:                   1.723
Prob(Omnibus):                  0.098   Jarque-Bera (JB):                2.945
Skew:                           0.579   Prob(JB):                        0.229
Kurtosis:                       4.087   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
# Independent Variables
X1_caut = data[['NSE ASI', 'FX Reserves']]

# Dependent Variable
Y2_caut = data['Cautious']


X1b_caut = sm.add_constant(X1_caut)
reg2_caut = sm.OLS(Y2_caut, X1b_caut).fit()
#prediction1 = reg1.predict(X1)

reg2_caut.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Cautious   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.369
Method:                 Least Squares   F-statistic:                     8.881
Date:                Wed, 22 Apr 2020   Prob (F-statistic):            0.00122
Time:                        19:38:18   Log-Likelihood:                 136.73
No. Observations:                  28   AIC:                            -267.5
Df Residuals:                      25   BIC:                            -263.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0112      0.000     27.547      0.000       0.010       0.012
NSE ASI         0.0172      0.006      2.939      0.007       0.005       0.029
FX Reserves     0.0254      0.009      2.973      0.006       0.008       0.043
==============================================================================
Omnibus:                        1.581   Durbin-Watson:                   1.100
Prob(Omnibus):                  0.454   Jarque-Bera (JB):                1.023
Skew:                          -0.468   Prob(JB):                        0.600
Kurtosis:                       2.965   Cond. No.                         23.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Balanced Portfolio*

In [30]:
Y3 = data['Balanced']


X1c = sm.add_constant(X1)
reg3 = sm.OLS(Y3, X1c).fit()
#prediction1 = reg1.predict(X1)

reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Balanced   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     1.126
Date:                Wed, 22 Apr 2020   Prob (F-statistic):              0.438
Time:                        19:38:18   Log-Likelihood:                 142.29
No. Observations:                  28   AIC:                            -248.6
Df Residuals:                      10   BIC:                            -224.6
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0115      0.001     17.171      0.000       0.010       0.013
Oil Prices          0.0030      0.010      0.304      0.768      -0.019       0.025
Inflation           0.0037      0.022      0.168      0.870      -0.045       0.053
NSE ASI             0.0202      0.009      2.158      0.056      -0.001       0.041
FX Reserves         0.0230      0.016      1.475      0.171      -0.012       0.058
USD/NGN (BDC)      -0.0266      0.014     -1.899      0.087      -0.058       0.005
USD/NGN (I&E)       0.0138      0.013      1.084      0.304      -0.015       0.042
91-Day Yield        0.0020      0.007      0.306      0.766      -0.013       0.017
182-Day Yield      -0.0006      0.012     -0.051      0.960      -0.026       0.025
364-Day Yield      -0.0066      0.015     -0.445      0.666      -0.039       0.026
2yr-Bond Yield     -0.0167      0.019     -0.899      0.390      -0.058       0.025
3yr-Bond Yield      0.0211      0.038      0.553      0.592      -0.064       0.106
5-yr-Bond Yield     0.0477      0.059      0.803      0.441      -0.085       0.180
7yr-Bond Yield     -0.1057      0.089     -1.192      0.261      -0.303       0.092
10yr-Bond Yield    -0.0191      0.107     -0.178      0.863      -0.258       0.220
12yr-Bond Yield     0.0815      0.198      0.412      0.689      -0.359       0.522
15yr-Bond Yield    -0.0340      0.139     -0.245      0.812      -0.344       0.276
20yr-Bond Yield     0.0360      0.072      0.497      0.630      -0.125       0.197
==============================================================================
Omnibus:                        2.850   Durbin-Watson:                   2.133
Prob(Omnibus):                  0.240   Jarque-Bera (JB):                1.663
Skew:                           0.572   Prob(JB):                        0.435
Kurtosis:                       3.340   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
# Independent Variables
X1_bal = data[['NSE ASI', 'USD/NGN (BDC)']]

# Dependent Variable
Y2_bal = data['Balanced']


X1b_bal = sm.add_constant(X1_bal)
reg2_bal = sm.OLS(Y2_bal, X1b_bal).fit()
#prediction1 = reg1.predict(X1)

reg2_bal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Balanced   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     4.661
Date:                Wed, 22 Apr 2020   Prob (F-statistic):             0.0190
Time:                        19:38:18   Log-Likelihood:                 131.75
No. Observations:                  28   AIC:                            -257.5
Df Residuals:                      25   BIC:                            -253.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0123      0.000     27.929      0.000       0.011       0.013
NSE ASI           0.0179      0.007      2.536      0.018       0.003       0.032
USD/NGN (BDC)    -0.0132      0.007     -2.025      0.054      -0.027       0.000
==============================================================================
Omnibus:                        0.435   Durbin-Watson:                   1.088
Prob(Omnibus):                  0.804   Jarque-Bera (JB):                0.555
Skew:                          -0.016   Prob(JB):                        0.758
Kurtosis:                       2.311   Cond. No.                         16.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Growth Portfolio*

In [32]:
Y4 = data['Growth']


X1d = sm.add_constant(X1)
reg4 = sm.OLS(Y4, X1d).fit()
#prediction1 = reg1.predict(X1)

reg4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Growth   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     4.075
Date:                Wed, 22 Apr 2020   Prob (F-statistic):             0.0142
Time:                        19:38:18   Log-Likelihood:                 100.12
No. Observations:                  28   AIC:                            -164.2
Df Residuals:                      10   BIC:                            -140.3
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0063      0.003      2.068      0.065      -0.000       0.013
Oil Prices          0.0179      0.044      0.407      0.693      -0.080       0.116
Inflation          -0.0518      0.099     -0.520      0.614      -0.273       0.170
NSE ASI             0.2627      0.042      6.220      0.000       0.169       0.357
FX Reserves        -0.0136      0.070     -0.194      0.850      -0.171       0.143
USD/NGN (BDC)      -0.0095      0.063     -0.151      0.883      -0.150       0.131
USD/NGN (I&E)       0.0859      0.057      1.501      0.164      -0.042       0.213
91-Day Yield        0.0026      0.030      0.087      0.933      -0.064       0.069
182-Day Yield      -0.0086      0.052     -0.166      0.872      -0.125       0.108
364-Day Yield      -0.0657      0.067     -0.987      0.347      -0.214       0.083
2yr-Bond Yield     -0.0825      0.084     -0.983      0.349      -0.270       0.105
3yr-Bond Yield     -0.0353      0.172     -0.205      0.842      -0.418       0.348
5-yr-Bond Yield    -0.0374      0.268     -0.140      0.892      -0.635       0.560
7yr-Bond Yield     -0.0265      0.400     -0.066      0.948      -0.918       0.865
10yr-Bond Yield     0.4843      0.484      1.000      0.341      -0.595       1.563
12yr-Bond Yield     0.0156      0.891      0.018      0.986      -1.970       2.001
15yr-Bond Yield     0.1068      0.627      0.170      0.868      -1.290       1.504
20yr-Bond Yield    -0.3583      0.326     -1.098      0.298      -1.085       0.369
==============================================================================
Omnibus:                        6.384   Durbin-Watson:                   3.153
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                5.207
Skew:                          -0.598   Prob(JB):                       0.0740
Kurtosis:                       4.741   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### Aggressive Portfolio

In [33]:
Y5 = data['Aggressive']


X1e = sm.add_constant(X1)
reg5 = sm.OLS(Y5, X1e).fit()
#prediction1 = reg1.predict(X1)

reg5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Aggressive   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     3.619
Date:                Wed, 22 Apr 2020   Prob (F-statistic):             0.0217
Time:                        19:38:18   Log-Likelihood:                 82.632
No. Observations:                  28   AIC:                            -129.3
Df Residuals:                      10   BIC:                            -105.3
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0019      0.006      0.328      0.749      -0.011       0.014
Oil Prices          0.0306      0.082      0.371      0.718      -0.153       0.214
Inflation          -0.0982      0.186     -0.528      0.609      -0.512       0.316
NSE ASI             0.4604      0.079      5.836      0.000       0.285       0.636
FX Reserves        -0.0433      0.132     -0.329      0.749      -0.336       0.250
USD/NGN (BDC)       0.0054      0.118      0.046      0.964      -0.258       0.269
USD/NGN (I&E)       0.1441      0.107      1.348      0.207      -0.094       0.382
91-Day Yield        0.0028      0.056      0.051      0.960      -0.121       0.127
182-Day Yield      -0.0156      0.098     -0.160      0.876      -0.233       0.202
364-Day Yield      -0.1130      0.124     -0.908      0.385      -0.390       0.164
2yr-Bond Yield     -0.1347      0.157     -0.859      0.411      -0.484       0.215
3yr-Bond Yield     -0.0813      0.321     -0.253      0.805      -0.797       0.634
5-yr-Bond Yield    -0.1126      0.501     -0.225      0.827      -1.228       1.003
7yr-Bond Yield      0.0499      0.747      0.067      0.948      -1.615       1.714
10yr-Bond Yield     0.8966      0.904      0.991      0.345      -1.119       2.912
12yr-Bond Yield    -0.0402      1.664     -0.024      0.981      -3.749       3.668
15yr-Bond Yield     0.2206      1.171      0.188      0.854      -2.389       2.830
20yr-Bond Yield    -0.6874      0.609     -1.128      0.285      -2.045       0.670
==============================================================================
Omnibus:                        5.027   Durbin-Watson:                   3.120
Prob(Omnibus):                  0.081   Jarque-Bera (JB):                3.517
Skew:                          -0.522   Prob(JB):                        0.172
Kurtosis:                       4.388   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Dollar Portfolio*

In [34]:
Y6 = data['Dollar']


X1f = sm.add_constant(X1)
reg6 = sm.OLS(Y6, X1f).fit()
#prediction1 = reg1.predict(X1)

reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Dollar   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                 -0.083
Method:                 Least Squares   F-statistic:                    0.8784
Date:                Wed, 22 Apr 2020   Prob (F-statistic):              0.609
Time:                        19:38:18   Log-Likelihood:                 116.23
No. Observations:                  28   AIC:                            -196.5
Df Residuals:                      10   BIC:                            -172.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0016      0.002      0.921      0.379      -0.002       0.005
Oil Prices          0.0131      0.025      0.526      0.610      -0.042       0.068
Inflation          -0.0578      0.056     -1.034      0.326      -0.183       0.067
NSE ASI             0.0350      0.024      1.474      0.171      -0.018       0.088
FX Reserves         0.0170      0.040      0.428      0.678      -0.071       0.105
USD/NGN (BDC)       0.0147      0.036      0.413      0.689      -0.065       0.094
USD/NGN (I&E)       0.0231      0.032      0.718      0.489      -0.049       0.095
91-Day Yield        0.0025      0.017      0.149      0.884      -0.035       0.040
182-Day Yield       0.0220      0.029      0.750      0.471      -0.043       0.087
364-Day Yield       0.0258      0.037      0.689      0.507      -0.058       0.109
2yr-Bond Yield      0.0018      0.047      0.037      0.971      -0.104       0.107
3yr-Bond Yield     -0.0034      0.097     -0.036      0.972      -0.219       0.212
5-yr-Bond Yield     0.0764      0.151      0.507      0.623      -0.260       0.412
7yr-Bond Yield      0.0379      0.225      0.169      0.869      -0.463       0.539
10yr-Bond Yield    -0.4285      0.272     -1.573      0.147      -1.035       0.178
12yr-Bond Yield     0.3311      0.501      0.660      0.524      -0.786       1.448
15yr-Bond Yield    -0.0400      0.353     -0.113      0.912      -0.826       0.746
20yr-Bond Yield    -0.0477      0.183     -0.260      0.800      -0.457       0.361
==============================================================================
Omnibus:                        3.245   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.197   Jarque-Bera (JB):                1.899
Skew:                          -0.597   Prob(JB):                        0.387
Kurtosis:                       3.453   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### **Monte Carlo Simulation**

In [0]:
stb_etf_returns1 = (new_prices['Stanbic ETF 30'].div(new_prices.groupby('month year')['Stanbic ETF 30'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_abs_returns1 = (new_prices['Stanbic Absolute Fund'].div(new_prices.groupby('month year')['Stanbic Absolute Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_gtd_returns1 = (new_prices['Stanbic Guaranteed Income Fund'].div(new_prices.groupby('month year')['Stanbic Guaranteed Income Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_euro_returns1 = (new_prices['Stanbic Eurobond Fund'].div(new_prices.groupby('month year')['Stanbic Eurobond Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
fbn_euro_returns1 = (new_prices['FBN Eurobond Fund'].div(new_prices.groupby('month year')['FBN Eurobond Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
axa_mmf_returns1 = axa_mmf_['AXA Money Market Fund Yield'].reset_index(drop = True)

In [0]:
month_returns_log = pd.DataFrame({'Date': month_returns['Date'], 
                                  'Stanbic ETF 30': stb_etf_returns1, 
                                  'Stanbic Absolute Fund': stb_abs_returns1, 
                                  'Stanbic Guaranteed Income Fund': stb_gtd_returns1, 
                                  'Stanbic Eurobond Fund': stb_euro_returns1, 
                                  'FBN Eurobond Fund': fbn_euro_returns1, 
                                  'AXA Money Market Fund Yield': axa_mmf_returns1})                                 

In [0]:
# Calculate the average return of each fund

stb_etf_mean = np.array(month_returns_log['Stanbic ETF 30'].mean())
stb_abs_mean = np.array(month_returns_log['Stanbic Absolute Fund'].mean())
stb_gtb_mean = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].mean())
stb_euro_mean = np.array(month_returns_log['Stanbic Eurobond Fund'].mean())
fbn_euro_mean = np.array(month_returns_log['FBN Eurobond Fund'].mean())
axa_mmf_mean = np.array(month_returns_log['AXA Money Market Fund Yield'].mean())

In [0]:
# Calculate the variance of each fund

stb_etf_var = np.array(month_returns_log['Stanbic ETF 30'].var())
stb_abs_var = np.array(month_returns_log['Stanbic Absolute Fund'].var())
stb_gtb_var = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].var())
stb_euro_var = np.array(month_returns_log['Stanbic Eurobond Fund'].var())
fbn_euro_var = np.array(month_returns_log['FBN Eurobond Fund'].var())
axa_mmf_var = np.array(month_returns_log['AXA Money Market Fund Yield'].var())

In [0]:
# Calculate the volatility of each fund

stb_etf_std = np.array(month_returns_log['Stanbic ETF 30'].std())
stb_abs_std = np.array(month_returns_log['Stanbic Absolute Fund'].std())
stb_gtb_std = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].std())
stb_euro_std = np.array(month_returns_log['Stanbic Eurobond Fund'].std())
fbn_euro_std = np.array(month_returns_log['FBN Eurobond Fund'].std())
axa_mmf_std = np.array(month_returns_log['AXA Money Market Fund Yield'].std())

In [0]:
# Calculate the drift of each fund

stb_etf_drift = np.array(stb_etf_mean - ( 0.5 * stb_etf_var))
stb_abs_drift = np.array(stb_abs_mean - ( 0.5 * stb_abs_var))
stb_gtb_drift = np.array(stb_gtb_mean - ( 0.5 * stb_gtb_var))
stb_euro_drift = np.array(stb_euro_mean - ( 0.5 * stb_euro_var))
fbn_euro_drift = np.array(fbn_euro_mean - ( 0.5 * fbn_euro_var))
axa_mmf_drift = np.array(axa_mmf_mean - ( 0.5 * axa_mmf_var))

In [0]:
t_intervals = 1200
n_iterations = 1000

In [0]:
stb_etf_pred = np.exp(stb_etf_drift + stb_etf_std * norm.ppf(np.random.normal(0.00423439, 0.06942789, (1200, 1000))))
stb_abs_pred = np.exp(stb_abs_drift + stb_abs_std * norm.ppf(np.random.normal(0.01246626, 0.00271151, (1200, 1000))))
stb_gtb_pred = np.exp(stb_gtb_drift + stb_gtb_std * norm.ppf(np.random.normal(0.01148387, 0.00601303, (1200, 1000))))
stb_euro_pred = np.exp(stb_euro_drift + stb_euro_std * norm.ppf(np.random.normal(0.0034685, 0.00692767, (1200, 1000))))
fbn_euro_pred = np.exp(fbn_euro_drift + fbn_euro_std * norm.ppf(np.random.normal(0.0021513, 0.01118711, (1200, 1000))))
axa_mmf_pred = np.exp(axa_mmf_drift + axa_mmf_std * norm.ppf(np.random.normal(0.01152736, 0.00242064, (1200, 1000))))

In [0]:
stb_etf_returns_list = np.zeros_like(stb_etf_pred)
stb_abs_returns_list = np.zeros_like(stb_abs_pred)
stb_gtb_returns_list = np.zeros_like(stb_gtb_pred)
stb_euro_returns_list = np.zeros_like(stb_euro_pred)
fbn_euro_returns_list = np.zeros_like(fbn_euro_pred)
axa_mmf_returns_list = np.zeros_like(axa_mmf_pred)

In [0]:
stb_etf_returns_list[0] = month_returns_log['Stanbic ETF 30'].iloc[-1]
stb_abs_returns_list[0] = month_returns_log['Stanbic Absolute Fund'].iloc[-1]
stb_gtb_returns_list[0] = month_returns_log['Stanbic Guaranteed Income Fund'].iloc[-1]
stb_euro_returns_list[0] = month_returns_log['Stanbic Eurobond Fund'].iloc[-1]
fbn_euro_returns_list[0] = month_returns_log['FBN Eurobond Fund'].iloc[-1]
axa_mmf_returns_list[0] = month_returns_log['AXA Money Market Fund Yield'].iloc[-1]

In [0]:
for t in range(1, t_intervals):
    stb_etf_returns_list[t] = stb_etf_returns_list[t - 1] * stb_etf_pred[t]
    stb_abs_returns_list[t] = stb_abs_returns_list[t - 1] * stb_abs_pred[t]
    stb_gtb_returns_list[t] = stb_gtb_returns_list[t - 1] * stb_gtb_pred[t]
    stb_euro_returns_list[t] = stb_euro_returns_list[t - 1] * stb_euro_pred[t]
    fbn_euro_returns_list[t] = fbn_euro_returns_list[t - 1] * fbn_euro_pred[t]
    axa_mmf_returns_list[t] = axa_mmf_returns_list[t - 1] * axa_mmf_pred[t]

In [0]:
future_dates = pd.date_range(start = '2018-12', periods = 1200, freq = 'M')
future_dates.strftime('%Y-%m')
index = 0

###### *Conservative Portfolio Simulation*

In [47]:
stb_a = stb_abs_returns_list * stb_abs1
axa_a = axa_mmf_returns_list * axa_mmf1

conserv = np.sum([stb_a, axa_a], axis = 0)
conservative = np.round(100000 + (conserv * 100000), 3)

conservative = pd.DataFrame(data = conservative[:, :])
conservative.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

conservative.tail(10)

Date            0            1            2            3  \
1190  2118-02  1253592.332  1241530.452  1244199.414  1238014.412   
1191  2118-03  1260155.682  1248199.840  1251094.473  1244988.499   
1192  2118-04  1267353.362  1255543.990  1257870.758  1252049.856   
1193  2118-05  1274156.483  1262863.480  1264551.807  1259090.585   
1194  2118-06  1281310.126  1270085.346  1271469.858  1266342.382   
1195  2118-07  1288452.179  1277052.731  1278750.330  1273453.197   
1196  2118-08  1295325.406  1284190.894  1285840.422  1280977.498   
1197  2118-09  1302400.087  1291391.258  1292931.549  1288395.273   
1198  2118-10  1309632.038  1298639.588  1300575.291  1295576.610   
1199  2118-11  1316645.225  1305843.720  1307907.671  1302832.792   

                4            5            6            7            8  \
1190  1246074.235  1253084.851  1241901.788  1239045.304  1254119.295   
1191  1252971.093  1259961.958  1248889.396  1246297.345  1261163.552   
1192  1260028.021  1266961.091  1255891.687  1253370.497  1267932.121   
1193  1266946.184  1274039.140  1263415.374  1259778.867  1275302.274   
1194  1274042.465  1281304.144  1270685.405  1266738.012  1282425.685   
1195  1281550.697  1288433.373  1277869.774  1273682.365  1289402.246   
1196  1288866.908  1295624.923  1285108.313  1280608.866  1296803.946   
1197  1296350.051  1302792.988  1292041.228  1288044.109  1304117.805   
1198  1303653.404  1310191.016  1299470.265  1295385.928  1311601.236   
1199  1311391.904  1317653.681  1306905.158  1303006.676  1319011.527   

                9           10           11           12           13  \
1190  1244142.359  1237380.796  1249314.944  1254587.484  1241280.589   
1191  1250917.331  1244266.813  1256090.200  1261695.532  1248287.838   
1192  1258004.566  1251181.205  1263234.988  1268862.417  1255581.115   
1193  1264969.465  1258272.215  1270207.837  1276044.666  1262495.101   
1194  1271849.749  1265173.674  1277470.176  1283326.615  1269556.289   
1195  1278996.979  1271937.345  1284768.154  1290319.924  1276746.809   
1196  1286046.989  1279211.208  1291950.183  1297702.457  1283787.392   
1197  1293378.567  1286511.507  1299459.591  1304895.344  1291362.733   
1198  1300248.803  1293494.592  1306712.238  1311852.931  1298316.357   
1199  1307393.160  1300455.946  1314113.885  1319237.272  1305759.037   

               14           15           16           17           18  \
1190  1233379.035  1244690.739  1235907.808  1249323.644  1242366.852   
1191  1240500.206  1251835.242  1242852.811  1256342.363  1249415.125   
1192  1247640.380  1258983.616  1250134.519  1263569.145  1256211.759   
1193  1254832.623  1266132.417  1257144.941  1270239.375  1263305.956   
1194  1261729.189  1273410.592  1264193.839  1277176.933  1270286.643   
1195  1268562.683  1280372.625  1271146.066  1284119.668  1277460.515   
1196  1275343.301  1287709.663  1278234.395  1290937.543  1284622.436   
1197  1282373.430  1294953.810  1285636.449  1298413.223  1291995.700   
1198  1289704.609  1302127.340  1293177.300  1305534.889  1298652.193   
1199  1296407.852  1309685.530  1300257.881  1312696.950  1305780.903   

               19           20           21           22           23  \
1190  1247238.575  1248910.052  1245510.016  1242874.643  1256517.627   
1191  1254357.766  1255958.340  1252666.079  1249992.509  1263804.594   
1192  1261232.028  1263148.667  1259520.910  1257350.886  1271046.253   
1193  1268325.446  1270124.106  1266767.317  1264369.245  1278199.458   
1194  1275521.717  1277374.227  1273864.448  1271272.790  1285425.081   
1195  1282849.064  1284663.674  1280600.598  1278407.940  1292235.979   
1196  1289790.516  1291659.305  1287766.573  1286004.291  1299301.284   
1197  1297034.543  1298531.292  1295107.876  1292877.696  1306765.988   
1198  1304302.131  1305818.163  1302738.087  1300168.402  1314143.034   
1199  1311808.526  1313306.388  1310129.932  1307462.545  1321676.761   

               24           25   

###### *Cautious Portfolio Simulation*

In [48]:
stb_b = stb_abs_returns_list * stb_abs2
stb_g = stb_gtb_returns_list * stb_GI
axa_b = axa_mmf_returns_list * axa_mmf2

caut = np.sum([stb_b, stb_g, axa_b], axis = 0)
cautious = np.round(100000 + (caut * 100000), 3)

cautious = pd.DataFrame(data = cautious[:, :])
cautious.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

cautious.head()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,2018-12,100839.096,100839.096,100839.096,100839.096,100839.096,100839.096,100839.096,100839.096,100839

###### *Balanced Portfolio Simulation*

In [49]:
stb_c = stb_abs_returns_list * stb_abs3
axa_c = axa_mmf_returns_list * axa_mmf3

bal = np.sum([stb_c, axa_c], axis = 0)
balanced = np.round(100000 + (bal * 100000), 3)

balanced = pd.DataFrame(data = balanced[:, :])
balanced.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

balanced.head()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,2018-12,100795.619,100795.619,100795.619,100795.619,100795.619,100795.619,100795.619,100795.619,100795

###### *Growth Portfolio Simulation*

In [50]:
stb_d1 = stb_abs_returns_list * stb_abs4
stb_d2 = stb_etf_returns_list * stb_etf1

grow = np.sum([stb_d1, stb_d2], axis = 0)
growth = np.round(100000 + (grow * 100000), 3)

growth = pd.DataFrame(data = growth[:, :])
growth.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

growth.head()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,2018-12,99848.193,99848.193,99848.193,99848.193,99848.193,99848.193,99848.193,99848.193,99848.193,9984

###### Aggressive Portfolio Simulation

In [51]:
stb_e1 = stb_abs_returns_list * stb_abs5
stb_e2 = stb_etf_returns_list * stb_etf2

aggres = np.sum([stb_e1, stb_e2], axis = 0)
aggressive = np.round(100000 + (aggres * 100000), 3)

aggressive = pd.DataFrame(data = aggressive[:, :])
aggressive.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

aggressive.tail()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,2118-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

###### *Dollar Portfolio Simulation*

In [52]:
stb_f = stb_euro_returns_list * stb_euro
fbn_a = fbn_euro_returns_list * fbn_euro

doll = np.sum([stb_f, fbn_a], axis = 0)
dollar = np.round(100000 + (doll * 100000), 3)

dollar = pd.DataFrame(data = dollar[:, :])
dollar.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

dollar.head()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,2018-12,99576.032,99576.032,99576.032,99576.032,99576.032,99576.032,99576.032,99576.032,99576.032,9957